In [ ]:
!pip install -r ../requirements.txt
print("··· Instalación en el entorno finalizada ···")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
files = {
    "df2015": "../data/2015.csv",
    "df2016": "../data/2016.csv",
    "df2017": "../data/2017.csv",
    "df2018": "../data/2018.csv",
    "df2019": "../data/2019.csv"
}

datasets = {}

for name, path in files.items():
    try:
        df = pd.read_csv(path, sep=",")
        datasets[name] = df
        print(f"{name} cargado correctamente ({len(df)} filas).")
    except Exception as e:
        print(f"Ha habido un error al cargar los datasets: {e}")

In [ ]:
datasets["df2015"].info()

In [ ]:
datasets["df2016"].info()

In [ ]:
datasets["df2017"].info()

In [ ]:
datasets["df2018"].info()

In [ ]:
datasets["df2019"].info()

Podemos comprobar que los datasets son bastante parecidos, tiene ligeras diferencias como la sintaxis del nombre de las columnas (Trust (Government Corruption) - Trust..Government.Corruption.) y que los datasets más actuales, 2018 y 2019 tienen menos columnas que años anteriores. Vamos comprobar si los tipos en las columnas siguen siendo iguales o han podido cambiar con el paso de los años.

In [ ]:
print("TIPOS DE DATOS DEL DATASET 2015")
print(datasets["df2015"].dtypes)
print("--------------------------------")
print("TIPOS DE DATOS DEL DATASET 2016")
print(datasets["df2016"].dtypes)
print("--------------------------------")
print("TIPOS DE DATOS DEL DATASET 2017")
print(datasets["df2017"].dtypes)
print("--------------------------------")
print("TIPOS DE DATOS DEL DATASET 2018")
print(datasets["df2018"].dtypes)
print("--------------------------------")
print("TIPOS DE DATOS DEL DATASET 2019")
print(datasets["df2019"].dtypes)
print("--------------------------------")

Vamos primero a renombrar las columnas para poder tratar mejor la información en su colectivo

In [ ]:
# 2015
datasets["df2015"] = datasets["df2015"].rename(columns={
    'Happiness Rank': 'Overall Rank', 
    'Happiness Score': 'Score', 
    'Economy (GDP per Capita)': 'GDP per Capita', 
    'Health (Life Expectancy)': 'Healthy Life Expectancy',
    'Trust (Government Corruption)': 'Perceptions of Corruption',
    'Freedom': 'Freedom to Make Life Choices',
    'Family': 'Social Support'
    })

In [ ]:
# 2016
datasets["df2016"] = datasets["df2016"].rename(columns={
    'Happiness Rank': 'Overall Rank', 
    'Happiness Score': 'Score', 
    'Economy (GDP per Capita)': 'GDP per Capita', 
    'Health (Life Expectancy)': 'Healthy Life Expectancy',
    'Trust (Government Corruption)': 'Perceptions of Corruption',
    'Freedom': 'Freedom to Make Life Choices',
    'Family': 'Social Support'
    })

In [ ]:
# 2017
datasets["df2017"] = datasets["df2017"].rename(columns={
    'Country': 'Country or region',
    'Happiness.Rank': 'Overall Rank',
    'Happiness.Score': 'Score', 
    'Economy..GDP.per.Capita.': 'GDP per Capita', 
    'Health..Life.Expectancy.': 'Healthy Life Expectancy',
    'Trust..Government.Corruption.': 'Perceptions of Corruption',
    'Freedom': 'Freedom to Make Life Choices',
    'Family': 'Social Support'
    })

Para empezar lo que voy a hacer es juntar la columna country con la columna región, ya que veo que en 2018 y 2019 existe la columna "Country or region", además de ordenar las columnas para tenerlas en el mismo orden que el resto de datasets

In [ ]:
media = datasets["df2015"]['Score'].mean()
error_estandar = datasets["df2015"]['Standard Error'].max()
print(f"Proporción para: df2015: ", error_estandar / media)  

# No es una proporción significativa, así que este campo lo eliminaremos del único dataset que lo tiene para homogenizar los datos

In [ ]:
# LIMPIEZA DE DATOS PARA EL 2015
datasets["df2015"]["Country or region"] = datasets["df2015"]["Country"] + " - " + datasets["df2015"]["Region"]
datasets["df2015"].insert(0, "Country or region", datasets["df2015"].pop("Country or region"))
datasets["df2015"].insert(0, "Overall Rank", datasets["df2015"].pop("Overall Rank"))
datasets["df2015"] = datasets["df2015"].drop(columns=["Country", "Region", "Dystopia Residual", "Standard Error"])
datasets["df2015"].head()

In [ ]:
# LIMPIEZA DE DATOS PARA EL 2016
datasets["df2016"]["Country or region"] = datasets["df2016"]["Country"] + " - " + datasets["df2016"]["Region"]
datasets["df2016"].insert(0, "Country or region", datasets["df2016"].pop("Country or region"))
datasets["df2016"].insert(0, "Overall Rank", datasets["df2016"].pop("Overall Rank"))
datasets["df2016"] = datasets["df2016"].drop(columns=["Country", "Region", "Dystopia Residual", "Lower Confidence Interval", "Upper Confidence Interval"])
datasets["df2016"].head()

In [ ]:
# LIMPIEZA DE DATOS PARA EL 2017
datasets["df2017"].insert(0, "Overall Rank", datasets["df2017"].pop("Overall Rank"))
datasets["df2017"] = datasets["df2017"].drop(columns=["Dystopia.Residual", "Whisker.high", "Whisker.low"])
datasets["df2017"].head()

Nuestros datasets ya están complemente igualados en cuestión de columnas, comprobemos una vez más los tipos, ahora se verá con más claridad

In [ ]:
for name, df in datasets.items():
    print(f"Tipos de columnas en {name}:")
    print(df.dtypes)
    print('-' * 40)

Empezaremos ahora comprobando los datos que tenemos

In [ ]:
for name, df in datasets.items():
    print(f"Comprobando valores nulos en {name}:")
    nulos_por_columna = datasets[name].isnull().sum()
    nulos_por_columna = nulos_por_columna[nulos_por_columna > 0]
    print(f"Valores nulos encontrados: {nulos_por_columna}")
    print('-' * 40)

Comprobando valores nulos en df2015:
Valores nulos encontrados: Series([], dtype: int64)
----------------------------------------
Comprobando valores nulos en df2016:
Valores nulos encontrados: Series([], dtype: int64)
----------------------------------------
Comprobando valores nulos en df2017:
Valores nulos encontrados: Series([], dtype: int64)
----------------------------------------
Comprobando valores nulos en df2018:
Valores nulos encontrados: Perceptions of corruption    1
dtype: int64
----------------------------------------
Comprobando valores nulos en df2019:
Valores nulos encontrados: Series([], dtype: int64)
----------------------------------------


In [33]:
#C Como solo tenemos un valor nulo en el dataset de 2018 vamos a echar un ojo
print("Posición: ", datasets["df2018"].isnull().stack().idxmax())
print("Fila del valor nulo: \n", datasets["df2018"].loc[19])

Posición:  (0, 'Overall rank')
Fila del valor nulo: 
 Overall rank                                      20
Country or region               United Arab Emirates
Score                                          6.774
GDP per capita                                 2.096
Social support                                 0.776
Healthy life expectancy                         0.67
Freedom to make life choices                   0.284
Generosity                                     0.186
Perceptions of corruption                      0.112
Name: 19, dtype: object


In [35]:
print(datasets["df2018"].shape)
# Hay 156 registros, utilizaremos la media de la columna para poder rellenarla
print("Media que se utilizará: ", datasets["df2018"]["Perceptions of corruption"].mean())
datasets["df2018"]["Perceptions of corruption"].fillna(datasets["df2018"]["Perceptions of corruption"].mean(), inplace=True)
print(datasets["df2018"].isnull().sum())

(156, 9)
Media que se utilizará:  0.112
Overall rank                    0
Country or region               0
Score                           0
GDP per capita                  0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
dtype: int64
